In [1]:
!python -V

Python 3.9.18


## Setup

In [12]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [4]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000/")
mlflow.set_experiment("nyc-taxi")

2023/11/24 09:48:21 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/1', creation_time=1700794101778, experiment_id='1', last_update_time=1700794101778, lifecycle_stage='active', name='nyc-taxi', tags={}>

## Data Preparation

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [17]:
train_data_path = "./data/green_tripdata_2021-01.parquet"
val_data_path = "./data/green_tripdata_2021-02.parquet"

df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Model Training

This is the simple demo how to use mlflow single run

In [18]:
from mlflow.data.pandas_dataset import PandasDataset

dataset_train: PandasDataset = mlflow.data.from_pandas(
    df_train, source=train_data_path
)
dataset_val: PandasDataset = mlflow.data.from_pandas(
    df_val, source=val_data_path
)

with mlflow.start_run():
    mlflow.set_tag("developer", "sugab")

    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", val_data_path)

    mlflow.log_input(dataset_train, context="train")
    mlflow.log_input(dataset_val, context="valid")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

2023/11/24 10:11:11 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Unable to map 'object' type to MLflow DataType. object can be mapped iff all values have identical data type which is one of (string, (bytes or byterray),  int, float).
2023/11/24 10:11:11 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Unable to map 'object' type to MLflow DataType. object can be mapped iff all values have identical data type which is one of (string, (bytes or byterray),  int, float).
